# Ray library for RL

`pip install ray[rllib]`<br>
On 2022-jun there was a problem with the latest version of the OpenAI Gym library. Such a command downgrades the version of this library.<br>
https://www.youtube.com/watch?v=krz8SCds7yA  полезная серия видио

In [7]:
import ray
ray.shutdown()
ray.init(ignore_reinit_error=True)

RayContext(dashboard_url='', python_version='3.8.10', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:54903', 'raylet_socket_name': 'tcp://127.0.0.1:60756', 'webui_url': '', 'session_dir': 'C:\\Users\\Admin\\AppData\\Local\\Temp\\ray\\session_2022-06-05_21-09-28_457296_5216', 'metrics_export_port': 54386, 'gcs_address': '127.0.0.1:57632', 'address': '127.0.0.1:57632', 'node_id': '7eb35983a81732198e63fb19ad646152f9d9c7cf3587b78722959cfe'})

## Choosing an environment

In [23]:
#ENV = "CartPole-v1"
ENV = "MountainCar-v0"
#ENV = "BipedalWalker-v3"

## Set up file locations for checkpoints

In [9]:
import shutil

CHECKPOINT_ROOT = "tmp/"+ENV
shutil.rmtree(CHECKPOINT_ROOT, ignore_errors=True, onerror=None)

RAY_RESULTS = "tmp/ray_results/"
shutil.rmtree(RAY_RESULTS, ignore_errors=True, onerror=None)

## Choose a method

- https://docs.ray.io/en/latest/rllib/rllib-algorithms.html - List of methods
- https://docs.ray.io/en/latest/rllib/rllib-training.html  - Common parameters

### 1. PPO: Proximal Policy Optimization   
<b>Actions</b> discrete:`Yes`, continuous:`Yes`<br>
Doesn't work for MountainCar-v0

In [5]:
import ray.rllib.agents.ppo as ppo

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

agent = ppo.PPOTrainer(config, env=ENV)

2022-06-03 19:42:41,656	INFO trainer.py:2295 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-06-03 19:42:41,658	INFO ppo.py:268 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-06-03 19:42:41,660	INFO trainer.py:864 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=24992) 2022-06-03 19:42:46,201	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictio

### 2. DQN: Deep Q-Network
<b>Actions</b> discrete:`Yes`, continuous:`No`<br>
Possible for MountainCar-v0

In [24]:
import ray.rllib.agents.dqn as dqn

config = dqn.DEFAULT_CONFIG.copy()
config['metrics_num_episodes_for_smoothing'] = 100    # def: 100 (для вычисления mean, min, max)
config["framework"]                          = 'tf'   # def: 'tf',  можно  'torch' 
print(config['model'])

agent = dqn.DQNTrainer(config, env=ENV)

2022-06-05 21:59:41,538	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).


{'_use_default_native_models': False, '_disable_preprocessor_api': False, '_disable_action_flattening': False, 'fcnet_hiddens': [256, 256], 'fcnet_activation': 'tanh', 'conv_filters': None, 'conv_activation': 'relu', 'post_fcnet_hiddens': [], 'post_fcnet_activation': 'relu', 'free_log_std': False, 'no_final_linear': False, 'vf_share_layers': True, 'use_lstm': False, 'max_seq_len': 20, 'lstm_cell_size': 256, 'lstm_use_prev_action': False, 'lstm_use_prev_reward': False, '_time_major': False, 'use_attention': False, 'attention_num_transformer_units': 1, 'attention_dim': 64, 'attention_num_heads': 1, 'attention_head_dim': 32, 'attention_memory_inference': 50, 'attention_memory_training': 50, 'attention_position_wise_mlp_dim': 32, 'attention_init_gru_gate_bias': 2.0, 'attention_use_n_prev_actions': 0, 'attention_use_n_prev_rewards': 0, 'framestack': True, 'dim': 84, 'grayscale': False, 'zero_mean': True, 'custom_model': None, 'custom_model_config': {}, 'custom_action_dist': None, 'custom_pr

2022-06-05 21:59:43,542	WARNING util.py:60 -- Install gputil for GPU system monitoring.


### 3. DDPG  (TD3):  Deep Deterministic Policy Gradients
<b>Actions</b> discrete:`No`, continuous:`Yes`<br>


In [20]:
import ray.rllib.agents.ddpg as ddpg

config = ddpg.DEFAULT_CONFIG.copy()
print(config['model'])

agent = ddpg.DDPGTrainer(config, env=ENV)

2022-06-05 21:57:17,026	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).


{'_use_default_native_models': False, '_disable_preprocessor_api': False, '_disable_action_flattening': False, 'fcnet_hiddens': [256, 256], 'fcnet_activation': 'tanh', 'conv_filters': None, 'conv_activation': 'relu', 'post_fcnet_hiddens': [], 'post_fcnet_activation': 'relu', 'free_log_std': False, 'no_final_linear': False, 'vf_share_layers': True, 'use_lstm': False, 'max_seq_len': 20, 'lstm_cell_size': 256, 'lstm_use_prev_action': False, 'lstm_use_prev_reward': False, '_time_major': False, 'use_attention': False, 'attention_num_transformer_units': 1, 'attention_dim': 64, 'attention_num_heads': 1, 'attention_head_dim': 32, 'attention_memory_inference': 50, 'attention_memory_training': 50, 'attention_position_wise_mlp_dim': 32, 'attention_init_gru_gate_bias': 2.0, 'attention_use_n_prev_actions': 0, 'attention_use_n_prev_rewards': 0, 'framestack': True, 'dim': 84, 'grayscale': False, 'zero_mean': True, 'custom_model': None, 'custom_model_config': {}, 'custom_action_dist': None, 'custom_pr

2022-06-05 21:57:18,842	WARNING util.py:60 -- Install gputil for GPU system monitoring.


### 4. SAC
<b>Actions</b> discrete:`Yes`, continuous:`Yes`<br>

In [7]:
import ray.rllib.agents.sac as sac

config = sac.DEFAULT_CONFIG.copy()

agent = sac.SACTrainer(config, env=ENV)

2022-06-03 19:53:19,905	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
2022-06-03 19:53:20,895	WARNING util.py:60 -- Install gputil for GPU system monitoring.


### 5. PG:  vanila  Policy Gradients
<b>Actions</b> discrete:`Yes`, continuous:`Yes`<br>
Ray: we include a vanilla policy gradients implementation as an example algorithm.

In [15]:
import ray.rllib.agents.pg as pg

config = pg.DEFAULT_CONFIG.copy()
config["framework"] = 'torch'

agent = pg.PGTrainer(config, env=ENV)


2022-06-05 21:16:51,601	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
2022-06-05 21:16:51,623	WARNING util.py:60 -- Install gputil for GPU system monitoring.


## Start training

In [18]:
N_ITER, file_name, episodes = 1000, "", 0
best_mean = -100000
for n in range(N_ITER):
    res = agent.train()    
    
    mean = res['episode_reward_mean']
    if mean > best_mean:
        file_name = agent.save(CHECKPOINT_ROOT)
        best_mean = mean

    episodes += res['episodes_this_iter']
    print(f"{n+1:3d} episodes:{episodes:5d}  mean:{mean:6.2f}  ({res['episode_reward_min']:6.2f}, {res['episode_reward_max']:6.2f})", file_name)


  1 episodes:   11  mean:-122.86  (-169.54, -110.17) tmp/BipedalWalker-v3\checkpoint_000002\checkpoint-2
  2 episodes:   13  mean:-123.59  (-169.54, -110.17) tmp/BipedalWalker-v3\checkpoint_000002\checkpoint-2
  3 episodes:   17  mean:-118.77  (-169.54, -89.92) tmp/BipedalWalker-v3\checkpoint_000004\checkpoint-4
  4 episodes:   38  mean:-110.95  (-169.54, -89.92) tmp/BipedalWalker-v3\checkpoint_000005\checkpoint-5
  5 episodes:   59  mean:-110.59  (-169.54, -89.92) tmp/BipedalWalker-v3\checkpoint_000006\checkpoint-6
  6 episodes:   80  mean:-114.47  (-169.54, -89.92) tmp/BipedalWalker-v3\checkpoint_000006\checkpoint-6
  7 episodes:  101  mean:-116.01  (-138.34, -89.92) tmp/BipedalWalker-v3\checkpoint_000006\checkpoint-6
  8 episodes:  121  mean:-118.25  (-126.65, -96.19) tmp/BipedalWalker-v3\checkpoint_000006\checkpoint-6
  9 episodes:  133  mean:-120.42  (-147.42, -96.19) tmp/BipedalWalker-v3\checkpoint_000006\checkpoint-6
 10 episodes:  154  mean:-123.34  (-147.42, -105.28) tmp/Biped

KeyboardInterrupt: 

## Model used
### For `config["framework"]  = 'tf'`

In [19]:
policy = agent.get_policy()
model = policy.model
print(model.base_model.summary())

AttributeError: 'NoopModel_as_DDPGTFModel' object has no attribute 'base_model'

### For `config["framework"]  = 'torch' `

In [19]:
policy = agent.get_policy()
model = policy.model
print(model)

FullyConnectedNetwork(
  (_logits): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=256, out_features=3, bias=True)
    )
  )
  (_hidden_layers): Sequential(
    (0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=2, out_features=256, bias=True)
        (1): Tanh()
      )
    )
    (1): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): Tanh()
      )
    )
  )
  (_value_branch): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=256, out_features=1, bias=True)
    )
  )
)


## Testing

In [ ]:
import numpy as np
import gym 
env = gym.make(ENV)                              # создать среду
 
def run(episodes = 1000, ticks = 10000):
    rews = []                                    # доходы за каждую попытку
    for _ in range(episodes):                    # делаем trials попыток (игр)
        tot = 0                                  # cуммарное вознаграждение
        obs = env.reset()                        # начальное состояние
        for _ in range(ticks):                   # ticks временных шагов
            action = agent.compute_single_action(obs)                 # выбрать действие
 
            obs, rew, done, _ = env.step(action) # получить информацию
            tot += rew                           # cуммарное вознаграждение            
            env.render()
            if done:                             
                break

        rews.append(tot)                         # накопить вознаграждение

    print(f"Reward: {np.mean(rews):.2f} ± {np.std(rews)/len(rews)**0.5:.2f}, std: {np.std(rews):.2f}, min: {np.min(rews)}, max: {np.max(rews)}")    
    
run(episodes = 10)    

## Loading the Model

In [ ]:
agent = dqn.DQNTrainer(config, env=ENV)
#agent = ppo.PPOTrainer(config, env=ENV)
agent.restore("tmp/MountainCar-v0\checkpoint_000120\checkpoint-120")
